In [1]:
############################################
###  Create Synthetic Data w. SMOTE API  ###
############################################

###################
### Credentials ###
###################

import os
import sys
from pathlib import Path

sys.path.append('C:/Users/chparr/OneDrive - SAS/credentials')
from credentials import hostname, session, port, protocol, wd, output_dir, git_dir, token_dir, token, token_refresh, token_pem

In [2]:
#############################
### Connect with SAS Viya ###
#############################

import swat

access_token = open(token, "r").read()
conn =  swat.CAS(hostname=hostname, username=None, password=access_token, ssl_ca_list=token_pem, protocol=protocol)
print(conn.serverstatus())

NOTE: Grid node action status report: 1 nodes, 9 total actions executed.
[About]

 {'CAS': 'Cloud Analytic Services',
  'CASCacheLocation': 'CAS Disk Cache',
  'CASHostAccountRequired': 'OPTIONAL',
  'Copyright': 'Copyright © 2014-2025 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2025-02-13T16:20:25Z',
  'System': {'Hostname': 'controller.sas-cas-server-default.innovationlab.svc.cluster.local',
   'Linux Distribution': 'Red Hat Enterprise Linux release 8.10 (Ootpa)',
   'Model Number': 'x86_64',
   'OS Family': 'LIN X64',
   'OS Name': 'Linux',
   'OS Release': '5.15.0-1078-azure',
   'OS Version': '#87-Ubuntu SMP Wed Dec 18 19:21:34 UTC 2024'},
  'Transferred': 'NO',
  'Version': '4.00',
  'VersionLong': 'V.04.00M0P01132025',
  'Viya Release': '20250130.1738270051125',
  'Viya Version': 'Stable 2025.01',
  'license': {'expires': '13Sep2026:00:00:00',
   'gracePeriod': 0,
   'site': 'CIS CREATE INNOVATION LAB (ENTERPRISE-RISK-MRM)',
   'siteNum': 70180938,
   'warningPeri

In [7]:
#############################
### Identify Table in CAS ###
#############################

### caslib and table to use in modeling
caslib = 'casuser'
in_mem_tbl = 'ANNUITY_ADVISORS_PREP'

### load table in-memory if not already exists in-memory
if conn.table.tableExists(caslib=caslib, name=in_mem_tbl).exists<=0:
    conn.table.loadTable(caslib=caslib, path=str(in_mem_tbl+str('.sashdat')), 
                         casout={'name':in_mem_tbl, 'caslib':caslib, 'promote':True})

### show table to verify
conn.table.tableInfo(caslib=caslib, wildIgnore=False, name=in_mem_tbl)

NOTE: Cloud Analytic Services made the file ANNUITY_ADVISORS_PREP.sashdat available as table ANNUITY_ADVISORS_PREP in caslib CASUSER(Chris.Parrish@sas.com).


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime,TableRedistUpPolicy
0,ANNUITY_ADVISORS_PREP,15351,33,0,utf-8,2025-02-13T16:25:42+00:00,2025-02-13T16:25:42+00:00,2025-02-13T16:25:42+00:00,UTF8,2.055083e+09,0,0,ANNUITY_ADVISORS_PREP.sashdat,CASUSER(Chris.Parrish@sas.com),0,Chris.Parrish@sas.com,,2024-10-23T06:42:10+00:00,2.045285e+09,Not Specified


In [8]:
caslib = 'casuser'
in_mem_tbl = 'annuity_advisors_prep'

conn.loadactionset(actionset="smote")
conn.smoteSample(table={"name":in_mem_tbl, 'caslib':caslib},
          numSamples=400000,
          classColumn="advisor_event_indicator",
          classToAugment=1,
          seed=10,
          casOut={"name":"annuity_advisors_prep_synthetic", 'caslib':caslib, "replace":True}
         )

NOTE: Added action set 'smote'.
NOTE: Adding 'classColumn': 'advisor_event_indicator' to the 'NOMINALS'.
NOTE: Smote completed successfully.


[ModelInfo]

             RowID                                        Description                    Value    nValue
 0      numSamples            Number of Synthetic Samples to Generate                   400000  400000.0
 1               k  Number of Nearest Neighbors to Use in Calculation                        5       5.0
 2            seed              Seed Used For Random Number generator                     10.0      10.0
 3     classColumn  Column Name in Input Table that Contains Class...  advisor_event_indicator       NaN
 4  classToAugment            Class to Generate Synthetic Samples For                        1       NaN

[NOBS]

       RowID                                 Description   Value
 0     NREAD                 Number of Observations Read    4606
 1     NUSED                 Number of Observations Used    4606
 2  NSAMPLES  Number of Synthetic Observations Generated  400000

+ Elapsed: 11.2s, user: 0.429s, sys: 1.03s, mem: 18.2mb